In [1]:
# Regular Funcs
import os
import cv2
import glob
import shutil

import pandas as pd
import pathlib
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image as ImagePIL
import plotly.express as px
import plotly.graph_objects as go

from numpy.random import randint
from sklearn.model_selection import train_test_split

In [2]:
# Statistics
from scipy import stats
from scipy import integrate

In [3]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential

from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
from keras.layers import Conv2DTranspose
from keras.layers import Reshape
from keras import backend

from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint

2023-07-03 09:03:31.338868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Custom Funcs
from Unpack_Scaffold_Data import readAndOutputDataset, curveVisualization

# Data Read Utility

In [5]:
curve_path = "/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN/scaffold_dataset_WU_LAB/Prints"
modulus_path = "/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN/scaffold_dataset_WU_LAB/Prints/modulus_data_types.csv"


In [6]:
X, y, y_df, file_order = readAndOutputDataset(curve_path, modulus_path, reverse=True);

DOC COUNT: 675
Operation Finished.

     Index     Modulus  Spacing  Infill  Height  Speed  Temperature   Mass  \
0        1  358.528888      0.8       1     0.1     30          190  0.394   
1        2  301.639039      0.9       1     0.1     30          190  0.334   
2        3  292.501492      1.0       1     0.1     30          190  0.308   
3        4  258.539802      1.1       1     0.1     30          190  0.286   
4        5  238.213024      1.2       1     0.1     30          190  0.259   
..     ...         ...      ...     ...     ...    ...          ...    ...   
670    671  151.559731      0.8       3     0.2     50          230  0.428   
671    672   85.074096      0.9       3     0.2     50          230  0.341   
672    673   52.285252      1.0       3     0.2     50          230  0.290   
673    674   70.811230      1.1       3     0.2     50          230  0.292   
674    675   36.627466      1.2       3     0.2     50          230  0.244   

     Porosity    Type  
0  

In [7]:
# Sanity Check
print("X SHAPE:", X.shape);
print("y SHAPE:", y.shape);
print();


# Visualization
# curveVisualization(X, y, file_order);

X SHAPE: (675, 2, 1803)
y SHAPE: (675, 10)



# Utility

In [8]:
def transposeStressData(X_Data):
    X = [];
    
    for data in X_Data:
        X.append(data.T);
        
    return np.array(X);

def normalizeStressStrain(x):
    for curve_index in range(len(x)):
        curve = x[curve_index];
        
        max_stress_val = np.max(curve[0]);
        max_strain_val = np.max(curve[1]);
        
        curve[0] = curve[0] / max_stress_val;
        curve[1] = curve[1] / max_strain_val;
        
    return x;

def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    
    Parameters
    -----------------
    x: Array of Homogenous (RGB) values of input data 
    
    Returns
    -----------------
    new_imgs: (numpy integer array) Numpy array of normalized data
    """
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

def stringtoCategorical(y):    
    data = [];
    
    for type_index in range(len(y)):
        wrd = y[type_index];
        encoding = 0.0;
        
        if (wrd == "Cubic"):
            encoding = 1.0;
        elif (wrd == "Gyroid"):
            encoding = 2.0;
            
        data.append([encoding]);
        
    return np.array(data);

# Process Parameter Stripping

In [9]:
def parameterStrip(y):
    y_t = y.T;
    
    Index = y_t[0];
    Modulus = y_t[1];
    Spacing = y_t[2];
    Infill = y_t[3];
    Height = y_t[4];
    Speed = y_t[5];
    Temp = y_t[6];
    Mass = y_t[7];
    Porosity = y_t[8];
    Type = y_t[9];
    return Index, Modulus, Spacing, Infill, Height, Speed, Temp, Mass, Porosity, Type

Index, Modulus, Spacing, Infill, Height, Speed, Temp, Mass, Porosity, Type = parameterStrip(y);

# Energy Absorption Calculation

Taking the integration under the curve!

In [10]:
Energy_Absorption = [];

for curve in X:
    interval_x = curve[0];
    interval_y = curve[1];
    
    val = integrate.simpson(interval_y, interval_x);
    Energy_Absorption.append(val);
    
Energy_Absorption = np.array(Energy_Absorption);

# Sanity Check
print(Energy_Absorption.shape);

(675,)


# Plotting: EDA

Here we take curve parameters and understand how the data is distributed and predicted. Understanding the data is half the problem!


In [ ]:
# 675 Stress-Strain Curve Domain
feature_domain_675 = list(range(675 + 1));
feature_domain_675.pop(0) 
feature_domain_675 = np.repeat(feature_domain_675, 4, axis=0) # Changed to 4

In [ ]:
feature_domain_8 = list(range(8 + 1));
feature_domain_8.pop(0);
feature_domain_8_rep = list(np.arange(1,9))*675

In [ ]:
feature_domain_7 = list(range(7 + 1));
feature_domain_7.pop(0);
feature_domain_7_rep = list(np.arange(1,8)) * 675

In [ ]:
feature_domain_4 = list(range(4 + 1));
feature_domain_4.pop(0);
feature_domain_4_rep = list(np.arange(1,5)) * 675

***NOTABLE FEATURES:***

For Scatter: 230C printing temperature has a void of Modulus values from 400-500 modulus values.

In [ ]:
# Modulus vs. Temperature
# X = Temperature
# Y = Modulus
fig = px.scatter(
    x=Temp, 
    y=Modulus,
    title="Each point of dataset over 675 curves",
    labels = {"x": "Temperature", "y": "Modulus"}
)
fig.show()

In [ ]:
# Modulus vs. Print Speed
# X = Temperature
# Y = Modulus
fig = px.scatter(
    x=Speed, 
    y=Modulus,
    title="Each point of dataset over 675 curves",
    labels = {"x": "Print Speed", "y": "Modulus"}
)
fig.show()

# Clustering

Each Clustering is based on the different characteristics of the curve. 

Most plots are a porosity vs Compression Modulus Plot, we see each variable how influential it is!

In [ ]:
Parameter_Sum_Str = [];

for index in range(len(Spacing)):
    Summation = Spacing[index] + Infill[index] + Height[index] + Speed[index] + Temp[index];
    Parameter_Sum_Str.append(str(Summation));
    
Parameter_Sum_Str = np.array(Parameter_Sum_Str);

In [ ]:
combination_str = [];

for curve in y:
    combination_str.append(str(curve));

combination_str = np.array(combination_str);

In [ ]:
Modulus_N = Modulus/np.max(Modulus)

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus_N,
    title="Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Temperature | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Temp
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Speed | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Speed
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Type | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Type
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Spacing | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Spacing
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Height | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Height
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="ALL | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=combination_str
)
fig.show()

In [ ]:
fig = px.scatter(
    x=Porosity, 
    y=Modulus,
    title="Parameter Sum | Porosity vs. Modulus",
    labels = {"x": "Porosity", "y": "Modulus"},
    text=Parameter_Sum_Str
)
fig.show()

### 3D Visualization of Modulus vs. Porosity vs. Energy Absorption

In [ ]:
df = px.data.iris()

fig = px.scatter_3d(
    x=Porosity, 
    y=Modulus, 
    z=Energy_Absorption, 
    color=Type
)
fig.show()

# Combination Encoding

The hard part of the problem is encapsulating the combination of variables to generate different types of combinations.

In [ ]:
cut_params_a = y[:, 1:2];
cut_params_b = y[:, 5:7];
cut_params_c = y[:, 8:9];
cut_params_combo = np.concatenate((cut_params_a, cut_params_b, cut_params_c), axis=1);

# Scale up to 10- 1000
for arr in cut_params_combo:
    arr[1] = arr[1] * 10
    arr[3] = arr[3] * 1000
    
print(cut_params_combo)


# Scaffold-GAN

### Dataset Manipulation

Let $D$ be a set of sets such that $|D| = 675$. Let $d_1, d_2, ..., d_n \in D$ be the set within sets. Printing scalar values that are set during the printing process. Suppose that $C$ and $P$ denote curve parameters and printing parameters respectively.

We noticed that infill type has the highest ***Spearman Correlation*** and ***Man Whitney U Sig***. We denote that for each $d_n$, there is a set of curve parameters $C_n$ and a set of printing parameters  $P_n$. We denote that $C_n = \{c_1, c_2, c_3\}$ where the parameters respectively represent ***Compression Modulus, Porosity, and Energy Absorption***. We denote that $P_n = \{p_1, p_2, p_3, p_4\}$ where the parameters respectively represent ***Spacing, Height, Speed, Temperature***. Note that $C_n, P_n \in d_n$.

An additional parameter to note, for **Infill Type**, the values are fixed discretely such that:

 $$Infill \ Type(P_2) = \begin{cases}
  0  & P_2 \text{ is Line} \\
  1  & P_2 \text{ is Cubic} \\
  2  & P_2 \text{ is Gyroid}
\end{cases}$$

In [ ]:
cut_params_1 = y[:, 1:2];
cut_params_2 = y[:, 8:9];

C_n = np.concatenate((cut_params_1, cut_params_2, (np.reshape(Energy_Absorption, (675,1)))), axis=1);

In [ ]:
print(C_n)

# Classification Problem

### Data Preparation & Split

In [ ]:
def integer_encoding(y):
    integer_encoding = [];
    
    for i in y:
        integer_encoding.append(i - 1);
    
    return np.array(integer_encoding);

y_label_categorical = integer_encoding(Infill);

y_label_categorical = tf.keras.utils.to_categorical(
    y_label_categorical, num_classes=3, dtype='float32'
);

print(y_label_categorical.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(C_n, y_label_categorical, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


print("X_train Shape:", X_train.shape);
print("y_train Shape:", y_train.shape);
print("X_val Shape:", X_val.shape);
print("y_val Shape:", y_val.shape);
print("X_test Shape:", X_test.shape);
print("y_test Shape:", y_test.shape);

### FC Classifier

In [ ]:
def InfillClassifier(in_shape = 3):
    in_parameters = tf.keras.Input(shape=in_shape)
    
    x = tf.keras.layers.Dense(10, input_dim=in_shape, activation='relu')(in_parameters)
    x = tf.keras.layers.Dense(5, activation='relu')(x)
    x = tf.keras.layers.Dense(3, activation='relu')(x)
    x = tf.keras.layers.Dense(3, activation='relu')(x)
    out = tf.keras.layers.Dense(3, activation='softmax')(x) # Output layer
    
    model = tf.keras.Model(in_parameters, out)
    
    opt = Adam(learning_rate =0.0001)
    model.compile(
        loss='categorical_crossentropy', 
        optimizer = opt, 
        metrics=['accuracy']
    );
    
    return model;


In [ ]:
infillModel = InfillClassifier();

infillModel.summary();

In [ ]:
n_epochs = 3000
X_train = X_train.astype('float32') # Convert to float32
X_val = X_val.astype('float32') # Convert to float32

history = infillModel.fit(
    X_train, y_train, batch_size=4, epochs=n_epochs, validation_data=(X_val, y_val)
)

### Loss Curves

In [ ]:
epochs_domain = list(range(n_epochs + 1))
epochs_domain.pop(0)


#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot()
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Test set prediction

In [ ]:
X_test = X_test.astype('float32') # Convert to float32
predict = infillModel.predict(X_test);

print("Samples:", len(X_test))
print()

incorrect_count = 0;

for predicted_index in range(len(predict)):
    arr = predict[predicted_index]
    truth = y_test[predicted_index]
    arr[arr < np.max(arr)] = 0
    arr[arr == np.max(arr)] = 1
    
    if (not np.array_equal(arr, truth)):
        incorrect_count += 1;
        
        print("Incorrectly predicted!")
        print("Expected:", truth)
        print("Predicted:", arr)
        print();
        
print("Incorrect:", str(int((incorrect_count / len(predict)) * 100)) + "%")
        
        
    

# Individualized GAN's

Now we see that given the infill type that best fits the stress-strain curve, what are the optimal parameters?

Recall that we denote that $P_n = \{p_1, p_2, p_3, p_4\}$ where the parameters respectively represent ***Spacing, Height, Speed, Temperature*** and $P_n \in d_n$. 

However, we want to  $C_n \cup P_n$. Thus, we apply each data to the GAN

### Seperate Data by Infill Type

In [ ]:
Line_Data = [];
Cubic_Data = [];
Gyroid_Data = [];

_y = cut_params = np.concatenate((
    y,
    (np.reshape(Energy_Absorption, (675,1))),
), axis=1);

for curve in _y:
    if ('Gyroid' in curve):
        Gyroid_Data.append(curve);
    elif ('Cubic' in curve):
        Cubic_Data.append(curve);
    elif ('Line' in curve):
        Line_Data.append(curve);
        
Line_Data = np.array(Line_Data);
Cubic_Data = np.array(Cubic_Data);
Gyroid_Data = np.array(Gyroid_Data);
      
# Sanity Check
print(Line_Data.shape)
print(Cubic_Data.shape)
print(Gyroid_Data.shape)


In [ ]:
print(y_df)

In [ ]:
def organizeParameters(_Data):
    Modulus = _Data[:, 1:2];
    Porosity = _Data[:, 8:9];
    Energy_Abs = _Data[:, 10:11];
    Spacing = _Data[:, 2:3];
    printing_params = _Data[:, 4:7];

    cut_params = np.concatenate((
        Modulus, 
        Porosity,
        Energy_Abs,
        Spacing,
        printing_params
    ), axis=1);
    
    return cut_params;

X_Line = organizeParameters(Line_Data);
X_Cubic = organizeParameters(Cubic_Data);
X_Gyroid = organizeParameters(Gyroid_Data);

# Sanity Check
print(X_Line.shape)
print(X_Cubic.shape)
print(X_Gyroid.shape)


### Plot Curves for some Visualization:

Note that it goes by: 

$$d_n =  \{ \text{Modulus}, \text{Porosity}, \text{Energy Absorption}, \text{Height}, \text{Distance}, \text{Speed}, \text{Temp} \} $$


In [ ]:
# # Single Lines Chart (DISTRIBUTION)
# fig_k = px.line(
#     x=feature_domain_7, 
#     y=X_Line[200],
#     title="Line: Single Parameter Curve",
#     labels={"x": "Parameters", "y":"Unnormalized values"}
# )

# fig_k.show()


# # Multiple Lines Chart (DISTRIBUTION)
# fig = go.Figure()

# for line in range(len(X_Line)):
#     data = X_Line[line];
#     fig.add_trace(go.Scatter(x=feature_domain_7, y=data))

# fig.show()

In [ ]:
# # Single Lines Chart (DISTRIBUTION)
# fig_k = px.line(
#     x=feature_domain_7, 
#     y=X_Cubic[200],
#     title="Cubic: Single Parameter Curve",
#     labels={"x": "Parameters", "y":"Unnormalized values"}
# )

# fig_k.show()


# # Multiple Lines Chart (DISTRIBUTION)
# fig = go.Figure()

# for line in range(len(X_Line)):
#     data = X_Cubic[line];
#     fig.add_trace(go.Scatter(x=feature_domain_7, y=data))

# fig.show()

In [ ]:
# # Single Lines Chart (DISTRIBUTION)
# fig_k = px.line(
#     x=feature_domain_7, 
#     y=X_Gyroid[200],
#     title="Cubic: Single Parameter Curve",
#     labels={"x": "Parameters", "y":"Unnormalized values"}
# )

# fig_k.show()


# # Multiple Lines Chart (DISTRIBUTION)
# fig = go.Figure()

# for line in range(len(X_Line)):
#     data = X_Gyroid[line];
#     fig.add_trace(go.Scatter(x=feature_domain_7, y=data))

# fig.show()

## Mode Specific Normalization

In [ ]:
# Skip for now...

In [ ]:
print("X_Line Max:", np.max(X_Line));
X_Line_N = X_Line / np.max(X_Line);

print("X_Cubic Max:", np.max(X_Cubic));
X_Cubic_N = X_Cubic / np.max(X_Cubic);

print("X_Gyroid Max:", np.max(X_Gyroid));
X_Gyroid_N = X_Gyroid / np.max(X_Gyroid);

In [ ]:
# Single Lines Chart (DISTRIBUTION)
fig_k = px.line(
    x=feature_domain_7, 
    y=X_Line_N[200],
    title="Line: Single Parameter Curve",
    labels={"x": "Parameters", "y":"Unnormalized values"}
)

fig_k.show()


# Multiple Lines Chart (DISTRIBUTION)
fig = go.Figure()

for line in range(len(X_Line_N)):
    data = X_Line_N[line];
    fig.add_trace(go.Scatter(x=feature_domain_7, y=data))

fig.show()

# Simple GAN

### Discriminator Data Sampling Generator

In [ ]:
def sample_real_samples(dataset, n_samples):
    """
    Parameters
    --------------
    real_dataset: dataset with the real data
    n_samples: amount of real images to sample from
    
    Returns
    --------------
    X: samples of n images in a list
    Y: labels of (1's) for true images (Binary Classification)
    """
    if (isinstance(dataset, list)):
        dataset = np.asarray(dataset);
        
    random_num = randint(0, dataset.shape[0], n_samples);
    X = dataset[random_num];
    y = np.ones((n_samples, 1));
    
    return X, y

### Discriminator

In [ ]:
def simpleDiscriminator(in_shape=7):
    """
    """
    model = tf.keras.Sequential();
    
    model.add(Dense(100, input_dim=in_shape, activation='relu')) 
    model.add(Dense(7, input_dim=in_shape, activation='relu')) 
    model.add(Dense(1, activation='sigmoid')) # Since the decision is binary (Real | Fake), we use sigmoid
    
    opt = Adam(learning_rate =0.001)
    model.compile(
        loss='binary_crossentropy', 
        optimizer = opt, 
        metrics=['accuracy']
    )
    
    return model

### Generator

In [ ]:
def simpleGenerator(in_shape=7):
    model = tf.keras.Sequential();
    
    model.add(Dense(7, input_dim=in_shape, activation="relu"))
    model.add(Dense(7, input_dim=in_shape, activation="relu"))
    model.add(Dense(7)) 
    
    return model;

### Summary of Models

In [ ]:
discriminator = simpleDiscriminator();
generator = simpleGenerator();

In [ ]:
discriminator.summary();
generator.summary();

### Latent Space

In [ ]:
def latentDimensionalGenerator(latent_dimensions, n_samples, randomGaussian = False):
    data = [];
    
    for sample in range(n_samples):
        x_input_0 = np.random.randn(latent_dimensions); # Points sampled from a normalized distribution.
        data.append(x_input_0);
        
    return np.array(data)

In [ ]:
# Generator production
def generate_samples(g_model, latent_dim, n_samples):
    x_input = latentDimensionalGenerator(latent_dim, n_samples)  # generate points in a latent space
    X = g_model.predict(x_input)
    y = np.zeros((n_samples, 1))  # create 'fake' class labels (0)
    return X, y

### Visualizing the latent dimensional space in 2D

In [ ]:
# k = latentDimensionalGenerator(7, 10)
# print(k)

fake_X, fake_y = generate_samples(generator, 7, 10);
print(fake_X.shape)

In [ ]:
for curve in fake_X:
    fig = go.Figure();
    fig.add_trace(go.Scatter(x=feature_domain_7, y=curve));
    fig.show();

### GAN: Putting it together

In [ ]:
def define_gan(generator, discriminator):
    discriminator.trainable = False # We set the discriminator as not trainable so the generator updates
    model = tf.keras.Sequential() 
    
    model.add(generator)
    model.add(discriminator)
    
    opt = Adam(learning_rate = 0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt) # Generator will train on this loss
    return model

# Evaluation Metrics

In [ ]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples, save_path=""):
    # Real Images based on discriminator
    X_real, y_real = sample_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    
    # Fake Images based on discriminator
    x_fake, y_fake = generate_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    
    print("============== CURVE GENERATION ON EPOCH", epoch,"==============");
    
    for curve in x_fake:
        plt.plot(feature_domain_7, curve)
    
    if (save_path != ""):
        plt.title("Training in epoch: " + str(epoch))
        plt.savefig(os.path.join(save_path, str(epoch) + '.png'));
        
    plt.show()
    
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100));

# GAN Training

In [ ]:
# train the generator and discriminator
def train_gan(g_model, d_model, gan_model, training_data, latent_dim, n_epochs, n_batch, save_path=""):
    d1Loss = [];
    d2Loss = [];
    gLoss = [];
    
    half_batch = int(n_batch / 2);
    
    for i in range(n_epochs):                
        # Real Image Discriminator Training
        X_real, y_real = sample_real_samples(training_data, half_batch)
        d_loss1, _ = d_model.train_on_batch(X_real, y_real) # Training on real

        # Fake Image Discriminator Training
        X_fake, y_fake = generate_samples(g_model, latent_dim, half_batch)
        d_loss2, _ = d_model.train_on_batch(X_fake, y_fake) # Training on fakes

        # Create a latent space and inverted labels
        X_gan = latentDimensionalGenerator(latent_dim, n_batch)
        y_gan = np.ones((n_batch, 1)) # Pretend that that they are all real.

        # Update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)

        # summarize loss on this batch
        print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
        summarize_performance(i, g_model, d_model, training_data, latent_dim, 100, save_path)
        
        d1Loss.append(d_loss1);
        d2Loss.append(d_loss2);
        gLoss.append(g_loss);
        
    return d1Loss, d2Loss, gLoss;

In [ ]:
latent_dim = 7;
gan_model = define_gan(generator, discriminator);

In [ ]:
# print(os.getcwd());
# os.chdir("/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN");
# image_save_path = "./images/"

# if not os.path.exists(image_save_path):
#     os.makedirs(image_save_path);

In [ ]:
n_epochs = 2000;
X_Line_N = X_Line_N.astype('float32')

#Training
d1, d2, gloss = train_gan(
    generator, 
    discriminator, 
    gan_model, 
    X_Line_N, 
    latent_dim, 
    n_epochs, # n_epochs
    10,  # batch size
);

# Losses